In [21]:
import pandas as pd
import numpy as np
import os, re
import warnings
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno
from collections import Counter
from datetime import datetime

from sklearn.metrics import mean_squared_error
%matplotlib inline

In [25]:
orig_pred_train = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\pred_train.csv")
orig_pred_test = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\pred_test.csv")
orig_pred = pd.concat([orig_pred_train, orig_pred_test], axis=0)
orig_pred = orig_pred.sort_values(by='index')

train_df = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\train_df_fe_all.csv")
sub_pred_train = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB\pred_train.csv")
sub_pred_test = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB\pred_test.csv")
sub_pred = pd.concat([sub_pred_train, sub_pred_test], axis=0)
sub_pred = sub_pred.sort_values(by='index')
sub_pred

,index,pred
90311,0,234777.203408
273853,1,422604.080399
96433,2,361897.716724
190472,3,148849.375001
2614,4,317098.251032
...,...,...
204863,431727,216892.970341
18849,431728,391794.226580
19131,431729,257210.786508
16464,431730,512921.279517


In [26]:
# for hdb data with main findings
orig_rmse = mean_squared_error(orig_pred['pred'], train_df.loc[orig_pred['index']]['resale_price'], squared=False)
sub_rmse = mean_squared_error(sub_pred['pred'], train_df.loc[sub_pred['index']]['resale_price'], squared=False)
print(f'original: {orig_rmse} vs subsampled: {sub_rmse} for full sized')

sub_ixes = train_df[(train_df['resale_year']>=2008) & (train_df['resale_year']<=2019)].index
orig_rmse = mean_squared_error(orig_pred[orig_pred['index'].isin(sub_ixes)]['pred'], train_df.loc[sub_ixes]['resale_price'], squared=False)
sub_rmse = mean_squared_error(sub_pred[sub_pred['index'].isin(sub_ixes)]['pred'], train_df.loc[sub_ixes]['resale_price'], squared=False)
print(f'original: {orig_rmse} vs subsampled: {sub_rmse} for sub sized')

original: 16941.299314719614 vs subsampled: 17352.333136236866 for full sized
original: 19061.092782277316 vs subsampled: 19580.454289055684 for sub sized


In [27]:
len(sub_ixes)

216670

In [4]:
test_df = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\test_df_fe_all.csv")
orig_pred = np.array(pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\submission.csv")['Predicted'])
sub_pred = np.array(pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB\submission.csv")['Predicted'])

final_ixes = test_df[(test_df['resale_year']>=2008) & (test_df['resale_year']<=2019)].index

print(min(orig_pred), np.mean(orig_pred), max(orig_pred))
orig_pred[final_ixes] = sub_pred[final_ixes]
print(min(orig_pred), np.mean(orig_pred), max(orig_pred))

final_pred = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\submission.csv")
final_pred['Predicted'] = orig_pred
final_pred.to_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB\submission_combined.csv", index=False)
final_pred.head()

42115.73604808684 301494.50997434626 1080483.221056197
42115.73604808684 301473.75739169866 1080483.221056197


,Id,Predicted
0,0,193164.816368
1,1,301922.681718
2,2,120992.464692
3,3,296164.496285
4,4,319119.716608


# WithHDB_WithoutOutliers

In [30]:
orig_pred_train = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\pred_train.csv")
orig_pred_test = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\pred_test.csv")
orig_pred = pd.concat([orig_pred_train, orig_pred_test], axis=0)
orig_pred = orig_pred.sort_values(by='index')

train_df = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\train_df_fe_all.csv")
sub_pred_train = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB_WithoutOutliers\pred_train.csv")
sub_pred_test = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB_WithoutOutliers\pred_test.csv")
sub_pred = pd.concat([sub_pred_train, sub_pred_test], axis=0)
sub_pred = sub_pred.sort_values(by='index')

sub_ixes = list(sub_pred['index'])
min_floor_area_sqm = min(train_df.loc[sub_ixes]['floor_area_sqm'])
max_floor_area_sqm = max(train_df.loc[sub_ixes]['floor_area_sqm'])
flat_type_list = list(train_df.loc[sub_ixes]['flat_type'].unique())
hdb_ixes = train_df[(train_df['resale_year']>=2008) & (train_df['resale_year']<=2019)].index

final_ixes = list(set(sub_ixes) & set(hdb_ixes))
orig_rmse = mean_squared_error(orig_pred[orig_pred['index'].isin(final_ixes)]['pred'], train_df.loc[final_ixes]['resale_price'], squared=False)
sub_rmse = mean_squared_error(sub_pred[sub_pred['index'].isin(final_ixes)]['pred'], train_df.loc[final_ixes]['resale_price'], squared=False)
print(f'original: {orig_rmse} vs subsampled: {sub_rmse} for sub sized')

original: 18987.256145057607 vs subsampled: 19530.40751962498 for sub sized


In [32]:
len(sub_pred)

430096

In [29]:
len(final_ixes)

215945

In [12]:
test_df = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\test_df_fe_all.csv")
orig_pred = np.array(pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\submission.csv")['Predicted'])
sub_pred = np.array(pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB_WithoutOutliers\submission.csv")['Predicted'])

sub_ixes = test_df[(test_df['floor_area_sqm']>= min_floor_area_sqm) & \
                    (test_df['floor_area_sqm']<= max_floor_area_sqm) & \
                    (test_df['flat_type'].isin(flat_type_list))
                   ].index
hdb_ixes = test_df[(test_df['resale_year']>=2008) & (test_df['resale_year']<=2019)].index
final_ixes = list(set(sub_ixes) & set(hdb_ixes))

print(min(orig_pred), np.mean(orig_pred), max(orig_pred))
orig_pred[final_ixes] = sub_pred[final_ixes]
print(min(orig_pred), np.mean(orig_pred), max(orig_pred))

final_pred = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\submission.csv")
final_pred['Predicted'] = orig_pred
final_pred.to_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB_WithoutOutliers\submission_combined.csv", index=False)
final_pred.head()

42115.73604808684 301494.50997434626 1080483.221056197
42115.73604808684 301480.61494389945 1080483.221056197


,Id,Predicted
0,0,193164.816368
1,1,301922.681718
2,2,120992.464692
3,3,305663.779651
4,4,319119.716608


# COMBINE ALL

In [24]:
orig_pred = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\submission.csv")
withhdb = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB\submission.csv")
nooutliers = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithoutOutliers\submission.csv")
withhdb_nooutliers = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_WithHDB_WithoutOutliers\submission_combined.csv")

In [25]:
withhdb = withhdb.rename(columns={'Predicted': 'WithHDB'})
nooutliers = nooutliers.rename(columns={'Predicted': 'WithoutOutliers'})
withhdb_nooutliers = withhdb_nooutliers.rename(columns={'Predicted': 'WithHDB_WithoutOutliers'})

for df in [withhdb, nooutliers, withhdb_nooutliers]:
    orig_pred = pd.merge(orig_pred, df, how='outer')

orig_pred.head()

,Id,Predicted,WithHDB,WithoutOutliers,WithHDB_WithoutOutliers
0,0,193164.816368,194243.552601,193271.844485,193164.816368
1,1,301922.681718,302489.887830,303758.126289,301922.681718
2,2,120992.464692,121688.468259,122013.203752,120992.464692
3,3,304289.111281,296164.496285,309138.697035,305663.779651
4,4,319119.716608,317206.696489,317562.281750,319119.716608


In [26]:
orig_pred['Final'] = [
    ww2 if ww2!=orig else (
        (wout+whdb)/2 if ((wout!=orig) & (whdb!=orig)) else (
            wout if wout!=orig else (
                whdb if whdb!=orig else orig))) for\
    whdb, wout, ww2, orig in zip(
        orig_pred['WithHDB'], orig_pred['WithoutOutliers'], orig_pred['WithHDB_WithoutOutliers'], orig_pred['Predicted'])
]

In [27]:
orig_pred.head()

,Id,Predicted,WithHDB,WithoutOutliers,WithHDB_WithoutOutliers,Final
0,0,193164.816368,194243.552601,193271.844485,193164.816368,193757.698543
1,1,301922.681718,302489.887830,303758.126289,301922.681718,303124.007060
2,2,120992.464692,121688.468259,122013.203752,120992.464692,121850.836005
3,3,304289.111281,296164.496285,309138.697035,305663.779651,305663.779651
4,4,319119.716608,317206.696489,317562.281750,319119.716608,317384.489120


In [28]:
orig_pred['Predicted']=orig_pred['Final']
orig_pred[['Id', 'Predicted']].to_csv(
    r"D:\29 CS5228\03 Project\HDBResalePrice\outs\lgb\0407_16497.04867\combined_ALL_submission.csv", index=False)